In [ ]:
%pip install pyannote.audio

In [2]:
# instantiate the pipeline
from pyannote.audio import Pipeline
from dotenv import load_dotenv
import os
import torch

load_dotenv()

access_token = os.getenv("HUGGINGFACE_TOKEN")

pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  token=access_token)


print(pipeline)

# CUDA를 사용할 수 있다면 CUDA를 사용하도록 설정 
if torch.cuda.is_available():
    pipeline.to(torch.device("cuda"))
    print("cuda is available")
else:
    print("cuda is not available")


cuda is not available


In [6]:
import librosa
import torch

path = "./audio/싼기타_비싼기타.mp3"

y, sample_rate = librosa.load(path, sr=16000, mono=True)
waveform = torch.tensor(y).float().unsqueeze(0)

print("shape:", waveform.shape, "sr:", sample_rate)
print("sec:", waveform.shape[-1] / sample_rate)
print("mean_abs:", waveform.abs().mean().item(), "max_abs:", waveform.abs().max().item())

diarize_out = pipeline({"waveform": waveform, "sample_rate": sample_rate})

# ✅ 여기 한 줄이 핵심 (너 버전에 맞는 Annotation)
ann = diarize_out.speaker_diarization
# (선택) 겹침 발화를 없애고 싶으면:
# ann = diarize_out.exclusive_speaker_diarization

with open("싼기타_비싼기타.rttm", "w", encoding="utf-8") as rttm:
    ann.write_rttm(rttm)


shape: torch.Size([1, 6967798]) sr: 16000
sec: 435.487375
mean_abs: 0.0471070222556591 max_abs: 0.9404489994049072
